In [1]:
import pandas as pd
import ast
import numpy as np

df = pd.read_csv("./data/UK_context0_llama.csv")
ddf = df.dropna(subset=["comment"])
future = ddf[ddf.comment.apply(lambda x: "future" in x)].index.tolist()
idx = df.index.tolist()
sentences = df.sentence.tolist()
pred = df.predictions.tolist()
pred = [ast.literal_eval(str(i)) if not isinstance(i, float) or not math.isnan(i) else [0 for ii in range(11)] for i in pred]


def convert_predictions(predictions, threshold=0.5):

    return [1 if p > threshold else 0 for p in predictions]
labels = []
for p in pred:
    labels.append(convert_predictions(p))

In [2]:
sentence_id_list = df.iloc[:,0].tolist()

In [3]:
statement_id_list = df.statement_id.tolist()


In [4]:
len(sentences)

2872

In [5]:
task_future_labels = []
for sentence, data, i in zip(sentences, np.array(labels), idx):    
    for ii, lbs in enumerate(data[3:]):
        if lbs == 1:
            if i in future:
                task_future_labels.append(1)
            else:
                task_future_labels.append(0)

In [6]:
sum(task_future_labels)

90

In [7]:
labels_name = [
    "approval",
    "signature",
    "c1 (reporting entity)",
    "c2 (structure)",
    "c2 (operations)",
    "c2 (supply chains)",
    "c3 (risk description)",
    "c4 (risk mitigation)",
    "c4 (remediation)",
    "c5 (effectiveness)",
    "c6 (consultation)"
]

In [8]:
import nltk
from collections import Counter

# Ensure required NLTK models are downloaded
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

def identify_tense_nltk(sentence):
    words = nltk.word_tokenize(sentence)
    tagged_words = nltk.pos_tag(words)
    tenses = {"past": 0, "future": 0}
    is_future = False

    # Expanded future-indicative words & phrases
    # future_keywords = [
    #     "will", "intend", "expect", "aim", "plan", "anticipate", "future",
    #     "going to", "shall", "may include", "plans to", "in 2024", "next year",
    #     "at a later stage", "commit to", "schedule", "set to", "forecast",
    #     "expected to", "proposed", "continue to", "implement", "develop",
    #     "undertake", "consider", "establish", "introduce", "expand",
    #     "launch", "explore", "assess", "enhance", "embed", "increase",
    #     "next steps", "to be completed"
    # ]
    future_keywords = [
        "will", "intend", "expect", "aim", "plan", "anticipate", "future",
        "going to", "shall", "may include", "plans to", "2024", "next year",
        "at a later stage", "next steps", "to be completed", "continue","continuing"
    ]

    # imperative_verbs = {"insert", "implement", "embed", "incorporate", "continue",
    #                     "recruit", "develop", "require", "ensure", "review",
    #                     "provide", "publish", "categorize", "train"}
    imperative_verbs = {}

    # Convert sentence to lowercase for easier keyword matching
    sentence_lower = sentence.lower()

    for i, (word, tag) in enumerate(tagged_words):
        # # Count past tense verbs
        # if tag == "VBD":
        #     tenses["past"] += 1
        # Detect future modal verbs
        if tag == "MD" and word.lower() in ["will", "shall"]:
            tenses["future"] += 1
            is_future = True
        # Identify infinitives that imply future actions (e.g., "to implement")
        elif tag == "TO" and i + 1 < len(tagged_words) and tagged_words[i + 1][1] == "VB":
            if tagged_words[i + 1][0].lower() in future_keywords:
                tenses["future"] += 1
                is_future = True
        if any(keyword in sentence_lower for keyword in future_keywords):
            tenses["future"] += 1
            is_future = True

    # Return "future" if future context was detected
    if is_future:
        return "future"

    # Determine the dominant tense based on counts
    #dominant_tense = max(tenses, key=tenses.get)
    return "no"


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/duoyizhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/duoyizhang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
import numpy as np
results = {key:[] for key in labels_name[3:]}
predicted_results = []
st = []
new_list = []
sen_id_list = []
stat_id_list = []
for sentence, data, i, si in zip(sentences, np.array(labels),sentence_id_list,statement_id_list):
    for idx, lbs in enumerate(data[3:]):
        if lbs == 1:
            result = identify_tense_nltk(sentence)
            if result == 'future':
                predicted_results.append(1)
            else:
                predicted_results.append(0)
            st.append(sentence)
            new_list.append(data)
            sen_id_list.append(i)
            stat_id_list.append(si)

In [10]:
len(new_list)

1247

In [11]:
pd.DataFrame({"sentence_id":sen_id_list, "statement_id":stat_id_list,"sentence":new_list, "prediction":predicted_results, "targets": task_future_labels}).to_csv("./future_negative_evidence_results/future_uk.csv", index = False)

In [12]:
correct = sum(t == p for t, p in zip(task_future_labels, predicted_results))
print("Overall accuracy is ", correct / len(task_future_labels))

Overall accuracy is  0.863672814755413


In [13]:
pos = np.array(task_future_labels)==1
correct = sum(t == p for t, p in zip(np.array(task_future_labels)[pos], np.array(predicted_results)[pos]))
print("Accuracy on prediction future works: ", correct / len(np.array(task_future_labels)[pos]))

Accuracy on prediction future works:  0.6666666666666666


In [14]:
import numpy as np
from sklearn.metrics import f1_score

print(f1_score(task_future_labels, predicted_results, average='micro')) 
print(f1_score(task_future_labels, predicted_results, average='weighted'))
print(f1_score(task_future_labels, predicted_results, average='macro'))

0.863672814755413
0.8861259763905242
0.6683303085299456
